# Example – Heat Transfer

## Introduction

Basically, the Fire Dynamics Simulator differentiates between two models for the calculation of heat conduction. While the one-dimensional case only calculates the temperature profile normal to the surface, the HT3D model also calculates lateral heat diffusion.
The 1D model is based on the one-cell method. For the coupling between solid and gas phase, it is important that the block of material is not wider than a cell, otherwise the heat will be transferred into a void phase with ambient temperature. Compared to the one-cell method, the HT3D model uses the multi-cell method. The representation of different material layers is done by apposition of different cells. For this task use the {download}`steel_beam_ht3d.fds` input file.


## 3D heat conduction - the HT3D model
Three-dimensional heat conduction in a solid obstruction can be invoked adding the `HT3D=.TRUE.` command in the respective `OBST` line. The heat transfer will automatically be calculated across several adjacent obstructions for that it is activated.

The material properties can be assigned to a solid obstruction by adding the `MATL_ID` on the `OBST` line or obtained from a certain `SURF_ID`. Note that the actual dimensions rather than the thermal `THICKNESS` of a `SURF` is taken into account for the computation of 3d heat transfer. For detailed information about the HT3D model please refer to section 11.3.9 of [FDS User's Guide](https://github.com/firemodels/fds/releases/download/FDS6.7.5/FDS_User_Guide.pdf).

## Setup
The extension of the computational domain is 

$$\small\sf [-0.8~m, 0.8~m] \times [0.0~m, 2.0~m] \times [0.0~m, 5.0~m]$$

:::{figure-md} fig-transfer-smv

<img src="figs/ht3d_setup.svg" width="100%">

SMV visualization of the geometry. The surface patch `HEATER`, which has a constant surface temperature of $1000^\circ C$ (`TMP_FRONT = 1000`) ,is colored redish. The orange and yellow `OBSTRUCTIONS` depict different surfaces (`SURF`). The gray heat shield covers thermocouples and devices from a radiative impact.  
:::

## Task
The commands `SOLID_PHASE_ONLY=.TRUE.` on the `MISC` line and `RADIATION = .False.` on the `RADI` line indicate that FDS turns off all gas phase computation as well as the RTE (Radiation Transport Equation) solver and therefore speeds up the computation. This can be useful if only the heat transfer in a solid obstruction is to be calculated.

***Task***
1. Implement the HT3D function in the given FDS input file and run the simulation for 3600 s. Evaluate the temperature within the solid obstructions at timesteps t = 100 s, 1000 s, 3600s by adding `SLCF` in the XZ and YZ plane(s). 
2. Create line devices at the center of each cell along the x-axis. Set `&DEVC...TIME HISTORY=.TRUE.` to get the temperature at each point with each time step. Alternative: Create a temperature SLCF at PBY = 0 and use the FDS READER module to load the data into a numpy array.
3. Evaluate the surface temperature as well as the of the solid obstructions via `BNDF` with quantity `WALL TEMPERATURE` as well as the 'RADIATIVE HEAT FLUX'
4. This task provides the same setup as in the previous tasks except that the heat source does not act as a constant surface temperature on the components but as a `VENT` with a predefined `HRR`. Replace the heat sources in the given FDS file with following lines:

    ```
    &REAC FUEL               = 'METHANOL'
          FYI                = 'Methanol C_1 H_4 O_1'
          CO_YIELD           = 0.001
          SOOT_YIELD         = 0.001 /

    &SURF ID='FIRE', COLOR='RED', HRRPUA=100./
    &VENT XB = -0.6,0.6, -0.1,0.1, 0.0,0.0, SURF_ID='FIRE' /
    ```

    Modify the FDS file in a way that the radiative as well as the convective heat transfer are considered. Again evaluate the `WALL TEMPERATURE` and the `RADIATIVE HEAT FLUX` as well as the `CONVECTIVE HEAT FLUX`.
    

In [1]:
import fdsreader
from fdsreader.bndf.utils import sort_patches_cartesian
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['lines.linewidth'] = 1

root = '../../../../'

In [2]:
data_root = root + 'data/heat_transfer/ht3d_1'
sim = fdsreader.Simulation(data_root)
slice_axial = sim.slices[0]
slice_steel = sim.slices[1]
slice_copper = sim.slices[2]
slice_concrete = sim.slices[3]

time = 3600
time_index = slice_steel.get_nearest_timestep(time)
vmin = 20
vmax = 1000

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(8,6), sharey=True)
heatmap = ax1.imshow(slice_steel[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
ax2.imshow(slice_copper[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
ax3.imshow(slice_concrete[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
ax1.set_title("Steel")
ax2.set_title("Copper")
ax3.set_title("Concrete")
fig.subplots_adjust(right=0.81)
cbar_ax = fig.add_axes([0.85, 0.17, 0.03, 0.67])
fig.colorbar(heatmap, cax=cbar_ax, label="Temperature $^\circ C$")
ax1.set_xlabel("$n_{cells}~X$")
ax2.set_xlabel("$n_{cells}~X$")
ax3.set_xlabel("$n_{cells}~X$")
ax1.set_ylabel("$n_{cells}~Z$")
plt.savefig('figs/beam_temperatures.svg', bbox_inches='tight')
plt.close()

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(10,10), sharex=True)
time_index = slice_axial.get_nearest_timestep(100)
ax1.imshow(slice_axial[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
time_index = slice_axial.get_nearest_timestep(1000)
ax2.imshow(slice_axial[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
time_index = slice_axial.get_nearest_timestep(3600)
ax3.imshow(slice_axial[0].data[time_index].T, origin='lower', cmap='jet',vmin=vmin, vmax=vmax)
ax1.set_title("T = 100 s")
ax2.set_title("T = 1000 s")
ax3.set_title("T = 3600 s")
ax1.set_ylabel("$n_{cells}~Z$")
ax2.set_ylabel("$n_{cells}~Z$")
ax3.set_ylabel("$n_{cells}~Z$")
ax3.set_xlabel("$n_{cells}~X$")
ax1.text(10, 5, "Steel", ha='center', color='white')
ax1.text(30, 5, "Copper", ha='center', color='white')
ax1.text(50, 5, "Concrete", ha='center', color='white')
ax2.text(10, 5, "Steel", ha='center', color='white')
ax2.text(30, 5, "Copper", ha='center', color='white')
ax2.text(50, 5, "Concrete", ha='center', color='white')
ax3.text(10, 5, "Steel", ha='center', color='white')
ax3.text(30, 5, "Copper", ha='center', color='white')
ax3.text(50, 5, "Concrete", ha='center', color='white')
fig.subplots_adjust(right=0.81)
cbar_ax = fig.add_axes([0.76, 0.13, 0.02, 0.75])
fig.colorbar(heatmap, cax=cbar_ax, label="Temperature $^\circ C$")
plt.savefig('figs/beam_temperatures_axial.svg', bbox_inches='tight')
plt.close()



:::{figure-md} ht3d-beam-temperatures

<img src="figs/beam_temperatures.svg" width="80%">

`SLCF` of beam `TEMPERATURE` for different `MATL` steel, 
:::

:::{figure-md} ht3d-beam-temperatures

<img src="figs/beam_temperatures_axial.svg" width="80%">

`SLCF` of beam `TEMPERATURE` for different `MATL` steel, 
:::

In [3]:
time_temp_df = pd.read_csv(data_root + "/steel_beam_ht3d_surf_false_devc.csv", skiprows=1, index_col=0)
time_temp_steel_df = time_temp_df.filter(regex='Steel')
time_temp_copper_df = time_temp_df.filter(regex='Copper')
time_temp_concrete_df = time_temp_df.filter(regex='Concrete')

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
ax1.imshow(time_temp_steel_df.T, aspect='auto', origin='lower', cmap='jet', vmin=20, vmax=800, extent=[0,3600,0,280])
ax2.imshow(time_temp_copper_df.T, aspect='auto', origin='lower', cmap='jet', vmin=20, vmax=800, extent=[0,3600,0,280])
ax3.imshow(time_temp_concrete_df.T, aspect='auto',origin='lower', cmap='jet', vmin=20, vmax=800, extent=[0,3600,0,280])
ax1.set_ylabel("Z / m")
# ax1.set_yticklabels([range(0,29, 4)])

ax2.set_ylabel("Z / mm")
ax3.set_ylabel("Z / mm")
ax3.set_xlabel("Time / s")
fig.subplots_adjust(right=0.81)
cbar_ax = fig.add_axes([0.85, 0.11, 0.03, 0.77])
fig.colorbar(heatmap, cax=cbar_ax, label="Temperature $^\circ C$")
plt.savefig('figs/temp_dev_copper.svg')
plt.close()

:::{figure-md} ht3d-beam-temperatures

<img src="figs/temp_dev_copper.svg" width="80%">

`SLCF` of beam `TEMPERATURE` for different `MATL` steel, 
:::

In [4]:
data_root = root + 'data/heat_transfer/HeatTransfer_0'
sim = fdsreader.Simulation(data_root)

def get_bndf_face(quantity, obstruction=0, orientation=-2):
    obst = sim.obstructions[0]
    patches = list()
    for sub_obst in obst.filter_by_orientation(orientation):
        # Get boundary data for a specific quantity
        sub_obst_data = sub_obst.get_data(quantity)
        patches.append(sub_obst_data.data[orientation])

        # Combine patches to a single face for plotting
        patches = sort_patches_cartesian(patches)

        shape_dim1 = sum([patch_row[0].shape[0] for patch_row in patches])
        shape_dim2 = sum([patch.shape[1] for patch in patches[0]])
        n_t = patches[0][0].n_t  # Number of timesteps

        face = np.empty(shape=(n_t, shape_dim1, shape_dim2))
        dim1_pos = 0
        dim2_pos = 0
        for patch_row in patches:
            d1 = patch_row[0].shape[0]
            for patch in patch_row:
                d2 = patch.shape[1]
                face[:, dim1_pos:dim1_pos + d1,
                dim2_pos:dim2_pos + d2] = patch.data
                dim2_pos += d2
            dim1_pos += d1
            dim2_pos = 0
    return face

total_hf = get_bndf_face('TOTAL HEAT FLUX')
convective_hf = get_bndf_face('CONVECTIVE HEAT FLUX')
radiative_hf = get_bndf_face('RADIATIVE HEAT FLUX')
gauge_hf = get_bndf_face('GAUGE HEAT FLUX')
incident_hf = get_bndf_face('INCIDENT HEAT FLUX')
vmin = 0
vmax = 80
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1,5,figsize=(15,30), sharey=True)
heatmap = ax1.imshow(total_hf[500].T, vmin=vmin, vmax=vmax, origin="lower", cmap="jet")
ax2.imshow(convective_hf[500].T, vmin=vmin, vmax=vmax, origin="lower", cmap="jet")
ax3.imshow(radiative_hf[500].T, vmin=vmin, vmax=vmax, origin="lower", cmap="jet")
ax4.imshow(incident_hf[500].T, vmin=vmin, vmax=vmax, origin="lower", cmap="jet")
ax5.imshow(gauge_hf[500].T, vmin=vmin, vmax=vmax, origin="lower", cmap="jet")
ax1.set_title("TOTAL HEAT FLUX")
ax2.set_title("CONVECTIVE HEAT FLUX")
ax3.set_title("RADIATIVE HEAT FLUX")
ax4.set_title("GAUGE HEAT FLUX")
ax5.set_title("INCIDENT HEAT FLUX")
ax1.set_xlabel("$n_{cells}~X$")
ax2.set_xlabel("$n_{cells}~X$")
ax3.set_xlabel("$n_{cells}~X$")
ax4.set_xlabel("$n_{cells}~X$")
ax5.set_xlabel("$n_{cells}~X$")
ax1.set_ylabel("$n_{cells}~Y$")

fig.subplots_adjust(right=0.83)
cbar_ax = fig.add_axes([0.85, 0.41, 0.02, 0.185])
cbar_ax.set_title('$kW / m^2$')
fig.colorbar(heatmap, cax=cbar_ax)
# ax5.colorbar(label="Temperature / $\sf ^\circ C$")
# plt.xlabel("$\sf N_{cells,x}$ / -")
# plt.ylabel("$\sf N_{cells,y}$ / -")
# fig.savefig('figs/heatflux_bndf.svg', bbox_inches='tight')
plt.close()